<a href="https://colab.research.google.com/github/NOVA0206/AI_ML_Projects/blob/main/book_recommendation/Book_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install numpy pandas scikit-learn

import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Function to fetch data from Google Books API
def fetch_books_data(query, max_results=40):
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults={max_results}"
    response = requests.get(url)
    books = response.json().get('items', [])
    return books

# Function to parse book data
def parse_books(books):
    book_data = []
    for book in books:
        volume_info = book.get('volumeInfo', {})
        book_data.append({
            'title': volume_info.get('title', 'N/A'),
            'authors': ', '.join(volume_info.get('authors', ['N/A'])),
            'description': volume_info.get('description', 'N/A'),
            'infoLink': volume_info.get('infoLink', 'N/A')
        })
    return book_data

# Fetch and parse book data
books = fetch_books_data("machine learning")
book_data = parse_books(books)

# Convert the data to a pandas DataFrame
df_books = pd.DataFrame(book_data)

# Display the first few rows of the DataFrame
print(df_books.head())

# Build a simple content-based recommendation system using TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_books['description'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get book recommendations
def get_recommendations(title, cosine_sim=cosine_sim):
    # Error handling for title not found in DataFrame
    if title not in df_books['title'].values:
        return "Title not found in the dataset"

    idx = df_books.index[df_books['title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get the scores of the 10 most similar books
    book_indices = [i[0] for i in sim_scores]
    return df_books.iloc[book_indices]

# Interactive loop to get recommendations
while True:
    user_input = input("Enter a book title to get recommendations (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    recommendations = get_recommendations(user_input)

    if isinstance(recommendations, str):
        print(recommendations)
    else:
        print("Recommended Books:")
        for idx, row in recommendations.iterrows():
            print(f"\nTitle: {row['title']}")
            print(f"Authors: {row['authors']}")
            print(f"Description: {row['description'][:200]}...")  # Print first 200 characters of the description
            print(f"More Info: {row['infoLink']}")

                              title  \
0                     Deep Learning   
1      Machine Learning For Dummies   
2  Encyclopedia of Machine Learning   
3  Introduction to Machine Learning   
4                  Machine Learning   

                                          authors  \
0  Ian Goodfellow, Yoshua Bengio, Aaron Courville   
1                John Paul Mueller, Luca Massaron   
2                 Claude Sammut, Geoffrey I. Webb   
3                                  Ethem Alpaydin   
4                                  Ethem Alpaydin   

                                         description  \
0  An introduction to a broad range of topics in ...   
1  Your no-nonsense guide to making sense of mach...   
2  This comprehensive encyclopedia, in A-Z format...   
3  Introduction -- Supervised learning -- Bayesia...   
4  A concise overview of machine learning—compute...   

                                            infoLink  
0  https://play.google.com/store/books/details?id...  